Universidad del valle de Guatemala  
Dpto. Ciencias de la computacion  
Inteligencia Artificial  
Alberto Suriano  

Laboratorio 8
Andres Quinto - 18288  
Marlon Hernández - 15177  

[Repositorio_aqui](https://github.com/AndresQuinto5/IA_LAB08.git)

### Tasks 1 - Teoría  

1. Investigar el **algoritmo AC-3** y su relación con el algoritmo de **backtracking search**  
    El **algoritmo AC-3** es un algoritmo de consistencia de arcos utilizado en problemas de satisfacción de restricciones (CSP). Su objetivo es reducir los dominios de las variables eliminando valores que no tienen soporte, es decir, que no pueden formar parte de una solución consistente con las restricciones. Por otro lado, **el algoritmo de backtracking search** es una técnica de búsqueda que intenta construir una solución paso a paso, retrocediendo cuando encuentra que una asignación de valores no lleva a una solución válida. La relación entre ambos es que **AC-3** puede utilizarse antes del **backtracking** para preprocesar el CSP, reduciendo los dominios y, por lo tanto, el número de asignaciones a probar durante el backtracking  

    Un ejemplo de este algoritmo podria ser, tenes una caja de lápices de colores y quieres asegurarte de que puedes dibujar un arcoíris completo. Pero hay una regla: cada color solo puede ir al lado de ciertos colores. **El algoritmo AC-3** es como un amigo que revisa todos los lápices y se asegura de que cada uno tenga un vecino adecuado antes de empezar a dibujar. Así, cuando comiences a colorear, no te detendrás a mitad de camino porque todos los lápices están en el orden correcto para hacer un arcoíris perfecto.

    **referencias:**
    - [Algoritmo AC-3](https://en.wikipedia.org/wiki/AC-3_algorithm)
    - [Backtracking Algorithms](https://www.freecodecamp.org/news/backtracking-algorithms-recursive-search/)

    
2. Defina en sus propias palabras el término “Arc Consistency”  
    **En mis propias palabras, “Arc Consistency”** se refiere a un estado en el que, para cada par de variables en un CSP que comparten una restricción, cada valor de la primera variable tiene al menos un valor correspondiente en la segunda variable que satisface la restricción entre ellas. Esto asegura que no hay valores aislados que hagan imposible encontrar una solución completa al problema.

    Un ejemplo podria un juego de parejas de cartas. Cada carta tiene un número y debes encontrarle una pareja que tenga el mismo número. **“Arc Consistency”** significa que todas las cartas tienen al menos una pareja posible. Si alguna carta no tuviera pareja, no podrías ganar el juego. Entonces, antes de jugar, revisas todas las cartas para asegurarte de que cada una tiene una pareja y así sabes que el juego se puede ganar.

    **referencias:**
    - [Arc Consistency](https://en.wikipedia.org/wiki/Arc_consistency)
    - [Arc Consistency Explained](https://www.boristhebrave.com/2021/08/30/arc-consistency-explained/)

### Task 2 - CSP con Backtracking, Beam y Local Search

In [55]:
class CSP:
    def __init__(self, variables, domains, constraints):
        self.variables = variables
        self.domains = domains
        self.constraints = constraints
        
    def is_consistent(self, assignment):
        for constraint in self.constraints:
            if not constraint(assignment):
                return False
        return True

Asignación de ejemplo:
E1: Lunes
E2: Martes
E3: Miércoles
E4: Lunes
E5: Martes
E6: Miércoles
E7: Lunes

¿La asignación cumple con las restricciones? False
